# Setup 

## Installing ModelScan

In [1]:
%pip install -q modelscan
!modelscan -v

Note: you may need to restart the kernel to use updated packages.
modelscan, version 0.0.0


In [2]:
%pip install -q torch==2.0.1
%pip install -q transformers==4.31.0
%pip install -q scipy==1.11.1

In [4]:
import torch
import os
from utils.pytorch_sentiment_model import download_model, predict_sentiment
from utils.pickle_codeinjection import PickleInject, get_payload

%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


# Saving Model


The BERT based sentiment analysis PyTorch model used in the notebook can be found at https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment. The safe model is saved at `./PyTorchModels/safe_model.pt`

In [5]:
# Save a model for sentiment analysis
from typing import Final

model_directory: Final[str] = "PyTorchModels"
if not os.path.isdir(model_directory):
    os.mkdir(model_directory)

safe_model_path = os.path.join(model_directory, "safe_model.pt")

download_model(safe_model_path)

# Safe Model Prediction

In [6]:
sentiment = predict_sentiment(
    "Stock market was bearish today", torch.load(safe_model_path)
)

The overall sentiment is: negative with a score of: 85.9%


# Scan Safe Model

The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no model serialization attacks, as expected.

In [7]:
!modelscan --path PyTorchModels/safe_model.pt

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/safe_model.pt:safe_model/data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

 No issues found! 🎉


# Model Serialization Attack

Here malicious code is injected in the safe model to read aws secret keys. The unsafe model is saved at `./PyTorchModels/unsafe_model.pt`

In [8]:
command = "system"
malicious_code = """cat ~/.aws/secrets
    """

unsafe_model_path = os.path.join(model_directory, "unsafe_model.pt")

payload = get_payload(command, malicious_code)
torch.save(
    torch.load(safe_model_path),
    f=unsafe_model_path,
    pickle_module=PickleInject([payload]),
)

# Unsafe Model Prediction

The malicious code injected in the unsafe model gets executed when it is loaded. The aws secret keys are displayed. 

Also, the unsafe model predicts the sentiments just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector. 


In [9]:
predict_sentiment("Stock market was bearish today", torch.load(unsafe_model_path))

aws_access_key_id=<access_key_id>
aws_secret_access_key=<aws_secret_key>
The overall sentiment is: negative with a score of: 85.9%


# Scan Unsafe Model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned. 

modelscan also outlines the found operator(s) and module(s) deemed unsafe. 

In [10]:
!modelscan --path  ./PyTorchModels/unsafe_model.pt

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan

--- Summary ---

Total Issues: 1

Total Issues By Severity:

    - LOW: 0
    - MEDIUM: 0
    - HIGH: 0
    - CRITICAL: 1

--- Issues by Severity ---

--- CRITICAL ---

Unsafe operator found:
  - Severity: CRITICAL
  - Description: Use of unsafe operator 'system' from module 'posix'
  - Source: /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl


# Reporting Format
ModelScan can report scan results in console (default), json, or custom report (to be defined by user in settings-file). For mode details, please see:  ` modelscan -h` 

## JSON Report

For JSON reporting: `modelscan -p ./path-to/file -r json -o output-file-name.json` 

In [11]:
# This will save the scan results in file: pytorch-model-scan-results.json
!modelscan --path  ./PyTorchModels/unsafe_model.pt -r json -o pytorch-model-scan-results.json

No settings file detected at /Users/mehrinkiani/Documents/modelscan/notebooks/modelscan-settings.toml. Using defaults. 

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl using modelscan.scanners.PickleUnsafeOpScan model scan
{"modelscan_version": "0.5.0", "timestamp": "2024-01-25T17:10:54.306065", 
"input_path": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt"
, "total_issues": 1, "summary": {"total_issues_by_severity": {"LOW": 0, 
"MEDIUM": 0, "HIGH": 0, "CRITICAL": 1}}, "issues_by_severity": {"CRITICAL": 
[{"description": "Use of unsafe operator 'system' from module 'posix'", 
"operator": "system", "module": "posix", "source": 
"/Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:
unsafe_model/data.pkl", "scanner": "modelscan.scanners.PickleUnsafeOpScan"}]}, 
"errors": [], "scanned": {"total_scanned": 1, "scanned_files": 
["/Users/mehrinkiani/Documents/modelscan/no